**initial setup**

In [ ]:
import gensim
import os
import itertools

wiki_path = 'D:\gensim_data\simplewiki'
wiki_prefix = 'gensim_'

# load id->word mapping (the dictionary), one of the results of step 2 above
id2word = gensim.corpora.Dictionary.load_from_text(os.path.join( wiki_path, wiki_prefix + 'wordids.txt.bz2'))

# load corpus iterator
mm = gensim.corpora.MmCorpus(os.path.join( wiki_path, wiki_prefix + 'tfidf.mm'))

# calculate LDA
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=400)

**get_topic_labels function**

In [ ]:
# Algorithm parameters
top_docs = 4       # how many documents to pick for phrases per topic
threshold = 0.03   # minimal tf-idf of a term to pick it as a part of a phrase

def get_topic_labels(corpus, lda_model, topicid, labeln=10):
    
    def _get_topic_in_doc( model, doc, topic):
        return abs(dict(model[doc]).get(topic, 0.0))
    
    # pick top_doc documents with the highest value (probablity) of topic topicid
    docs = sorted(mm, reverse=True, key=lambda doc: _get_topic_in_doc(lda, doc, topicid))[:top_docs]
    
    # construct two-term phrases to serve label candidates
    phrases = []
    for doc in docs:
        # pick only significant tokens
        tokens = [t[0] for t in doc if t[1]>=threshold]

        # collect combinations of pairs of terms picked on the previous step
        phrases.extend(itertools.combinations(tokens,2))
        
    # eliminate duplicate phrases coming from different documents
    phrases = set(phrases)
    # sort phrases according to value (probablity) of topic topicid
    phrases = sorted(phrases, reverse=True, key=lambda phrase: _get_topic_in_doc(lda, [(phrase[0],1),(phrase[1],1)], topicid))

    highest_coherence = _get_topic_in_doc(lda, [(phrases[0][0],1),(phrases[0][1],1)], topicid)

    # return top candiadates with the same score as the highest one, limited by labeln
    candidates = []
    for i, phrase in enumerate(phrases):
        if i >= labeln:
            break
        a, b = phrase
        if get_topic_in_doc(lda, [(a,1),(b,1)], topic) == highest_coherence:
            candidates.append(''.join([id2word[a], id2word[b]]))

    return candidates

**running the finction**

In [ ]:
topicid = 15
print(get_topic_labels(mm, lda, topicid))